In [1]:
import os
import sys
sys.path.append('../')

from rocket_rag.node_indexing import *

c:\Users\HAOXUAN\miniconda3\envs\agent\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


In [3]:
ts_node_indexer = TimeSeriesNodeIndexer()

TypeError: TimeSeriesNodeIndexer.__init__() missing 1 required positional argument: 'ts_transform'